In [ ]:
# Import thư viện và khởi tạo
import ee
import geemap
import os
import numpy as np
from datetime import datetime

# Khởi tạo Google Earth Engine với project ID
ee.Initialize(project='ee-bonglantrungmuoi')

# Cấu hình thời gian và tham số
start_date = '2022-01-01'
end_date = '2025-01-01'

# Tạo thư mục lưu dữ liệu
output_folder = './chirps_data'
os.makedirs(output_folder, exist_ok=True)

print(f"=== CẤU HÌNH TẢI DỮ LIỆU CHIRPS ===")
print(f"- Thời gian: {start_date} đến {end_date}")
print(f"- Dataset: CHIRPS Precipitation Monthly")
print(f"- Thư mục lưu: {output_folder}")
print(f"- Sử dụng: geemap cho file lớn")
print(f"- Mục đích: Nghiên cứu độ ẩm đất nông nghiệp")

In [ ]:
# Đọc shapefile và tạo CHIRPS collection
print("=== TẢI SHAPEFILE VÀ TẠO CHIRPS COLLECTION ===")

# Sử dụng shapefile Lâm Đồng từ GEE Asset
lamdong_asset = ee.FeatureCollection('projects/ee-bonglantrungmuoi/assets/lamdong')
lamdong_geometry = lamdong_asset.geometry()

# Hiển thị thông tin về shapefile
print(f"✓ Đã tải shapefile từ GEE Asset")
print(f"  - Số features: {lamdong_asset.size().getInfo()}")

# Tạo CHIRPS collection
# Sử dụng CHIRPS Daily để tạo composite monthly
# CHIRPS Daily: 'UCSB-CHG/CHIRPS/DAILY' (độ phân giải 0.05 degree ~ 5.5km)
chirps_collection = (ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')
                    .filterBounds(lamdong_geometry)
                    .filterDate(start_date, end_date)
                    .select('precipitation'))

# Thông tin collection
collection_size = chirps_collection.size().getInfo()
print(f"✓ Đã tạo CHIRPS collection")
print(f"  - Tổng số ảnh: {collection_size}")

# Thông tin ảnh đầu tiên
if collection_size > 0:
    first_image = ee.Image(chirps_collection.first())
    first_date = ee.Date(first_image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
    print(f"  - Ảnh đầu tiên: {first_date}")
    
    # Thống kê theo năm
    for year in range(2022, 2025):
        year_collection = chirps_collection.filter(ee.Filter.calendarRange(year, year, 'year'))
        year_count = year_collection.size().getInfo()
        print(f"  - Năm {year}: {year_count} ảnh")
        
    # Hiển thị giá trị mẫu
    sample_stats = first_image.reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=lamdong_geometry,
        scale=5500,
        maxPixels=1e9
    )
    print(f"  - Giá trị mẫu (mm): {sample_stats.getInfo()}")
else:
    print("⚠️ Không tìm thấy ảnh nào!")

In [ ]:
# Functions tải dữ liệu CHIRPS với geemap
print("=== THIẾT LẬP FUNCTIONS TẢI DỮ LIỆU CHIRPS ===")

def download_monthly_chirps(year, month, scale=5500):
    """
    Tải dữ liệu CHIRPS tháng với geemap (từ CHIRPS Daily)
    
    Args:
        year: Năm
        month: Tháng  
        scale: Độ phân giải (m), mặc định 5500m (native CHIRPS resolution)
    """
    start_month = ee.Date.fromYMD(year, month, 1)
    end_month = start_month.advance(1, 'month')
    
    # Lọc ảnh theo tháng từ CHIRPS Daily
    monthly_collection = chirps_collection.filterDate(start_month, end_month)
    
    # Kiểm tra có dữ liệu không
    count = monthly_collection.size().getInfo()
    if count == 0:
        print(f"❌ Không có dữ liệu cho {year}-{month:02d}")
        return None
    
    print(f"📊 Tháng {year}-{month:02d}: {count} ảnh daily")
    
    # Tạo composite monthly từ dữ liệu daily (tổng lượng mưa trong tháng)
    monthly_precipitation = (monthly_collection
                           .select('precipitation')
                           .sum()  # Tổng lượng mưa trong tháng
                           .clip(lamdong_geometry))
    
    # Tạo tên file
    filename = f'CHIRPS_Lamdong_{year}_{month:02d}_monthly.tif'
    filepath = os.path.join(output_folder, filename)
    
    try:
        print(f"🔄 Đang tải {filename}...")
        
        # Sử dụng geemap để tải file
        geemap.download_ee_image(
            image=monthly_precipitation,
            filename=filepath,
            scale=scale,
            region=lamdong_geometry,
            crs='EPSG:4326'
        )
        
        # Kiểm tra file size
        if os.path.exists(filepath):
            file_size = os.path.getsize(filepath) / (1024*1024)  # MB
            print(f"✅ Đã lưu: {filename} ({file_size:.1f} MB)")
            return filepath
        else:
            print(f"❌ Không tạo được file: {filename}")
            return None
        
    except Exception as e:
        print(f"❌ Lỗi tải {year}-{month:02d}: {str(e)}")
        return None

def get_chirps_stats(year, month):
    """
    Lấy thống kê mưa cho tháng (từ CHIRPS Daily)
    """
    start_month = ee.Date.fromYMD(year, month, 1)
    end_month = start_month.advance(1, 'month')
    
    monthly_collection = chirps_collection.filterDate(start_month, end_month)
    
    if monthly_collection.size().getInfo() == 0:
        return None
    
    # Tạo composite monthly từ dữ liệu daily
    monthly_precipitation = monthly_collection.select('precipitation').sum()
    
    # Tính thống kê trong vùng Lâm Đồng
    stats = monthly_precipitation.reduceRegion(
        reducer=ee.Reducer.mean().combine(
            ee.Reducer.minMax(), sharedInputs=True
        ).combine(
            ee.Reducer.stdDev(), sharedInputs=True
        ),
        geometry=lamdong_geometry,
        scale=5500,
        maxPixels=1e9
    )
    
    return stats.getInfo()

def estimate_chirps_file_size(scale=5500):
    """Ước tính kích thước file CHIRPS"""
    try:
        # Tạo composite mẫu từ 1 tháng
        sample_start = ee.Date('2022-01-01')
        sample_end = sample_start.advance(1, 'month')
        
        sample_collection = chirps_collection.filterDate(sample_start, sample_end)
        sample_image = sample_collection.sum()
        
        pixel_count = (sample_image.select('precipitation')
                      .reduceRegion(
                          reducer=ee.Reducer.count(),
                          geometry=lamdong_geometry,
                          scale=scale,
                          maxPixels=1e9
                      ).get('precipitation').getInfo())
        
        # Ước tính: 1 band × 4 bytes/pixel (Float32)
        size_mb = (pixel_count * 4) / (1024 * 1024)
        
        print(f"📏 Ước tính kích thước file CHIRPS (độ phân giải {scale}m):")
        print(f"   - Số pixel: {pixel_count:,}")
        print(f"   - Kích thước: {size_mb:.2f} MB")
        
        return size_mb
        
    except Exception as e:
        print(f"❌ Không thể ước tính: {str(e)}")
        return 0

print("✅ Đã thiết lập functions tải dữ liệu CHIRPS với geemap!")

In [ ]:
# TẢI DỮ LIỆU CHIRPS THÁNG (2022-2024)
print("=== TẢI DỮ LIỆU CHIRPS PRECIPITATION THÁNG ===")
print("Thời gian: Tháng 1/2022 → Tháng 12/2024")
print("Dữ liệu: CHIRPS Daily → Monthly Composite (mm)")
print("Độ phân giải: 5.5km (~0.05 degree)")
print("Phương pháp: Tổng lượng mưa daily trong tháng")

# Cấu hình
SCALE = 5500  # Độ phân giải native của CHIRPS
total_months = 3 * 12  # 3 năm × 12 tháng = 36 tháng

# Ước tính kích thước file
print(f"\n🔍 Ước tính kích thước với độ phân giải {SCALE}m:")
estimated_size = estimate_chirps_file_size(SCALE)

print(f"📊 Tổng quan:")
print(f"   - Số tháng cần tải: {total_months} tháng")
print(f"   - Kích thước ước tính/file: {estimated_size:.2f} MB")
print(f"   - Tổng dung lượng ước tính: {(estimated_size * total_months):.1f} MB ({(estimated_size * total_months)/1024:.2f} GB)")

# Hiển thị một số thống kê mẫu
print(f"\n📈 Thống kê mẫu (tháng 1/2022):")
sample_stats = get_chirps_stats(2022, 1)
if sample_stats:
    precip_stats = sample_stats.get('precipitation_mean', 0)
    print(f"   - Lượng mưa tổng: {precip_stats:.1f} mm/tháng")

print(f"\n🚀 BẮT ĐẦU TẢI {total_months} FILE CHIRPS THÁNG...")

downloaded_files = []
failed_months = []
count = 0

# Tải từng tháng từ 2022-2024
for year in range(2022, 2025):  # 2022, 2023, 2024
    for month in range(1, 13):  # Tháng 1-12
        count += 1
        
        print(f"\n[{count}/{total_months}] Tháng {year}-{month:02d}")
        
        try:
            # Hiển thị thống kê trước khi tải
            stats = get_chirps_stats(year, month)
            if stats and 'precipitation_mean' in stats:
                avg_precip = stats['precipitation_mean']
                min_precip = stats.get('precipitation_min', 0)
                max_precip = stats.get('precipitation_max', 0)
                print(f"   📊 Tổng lượng mưa: {avg_precip:.1f}mm (min: {min_precip:.1f}, max: {max_precip:.1f})")
            
            # Tải file
            filepath = download_monthly_chirps(year, month, SCALE)
            
            if filepath:
                downloaded_files.append(filepath)
                print(f"✅ Thành công: {os.path.basename(filepath)}")
            else:
                failed_months.append(f"{year}-{month:02d}")
                print(f"❌ Thất bại: {year}-{month:02d}")
                
        except Exception as e:
            failed_months.append(f"{year}-{month:02d}")
            print(f"❌ Lỗi {year}-{month:02d}: {str(e)}")
        
        # Hiển thị tiến trình
        progress = (count / total_months) * 100
        print(f"📈 Tiến trình: {progress:.1f}% ({count}/{total_months})")

# Tổng kết
print(f"\n{'='*50}")
print(f"🎉 HOÀN TẤT QUÁ TRÌNH TẢI CHIRPS!")
print(f"{'='*50}")

if downloaded_files:
    total_size = sum(os.path.getsize(f) for f in downloaded_files) / (1024*1024)
    
    print(f"✅ Thành công:")
    print(f"   - Đã tải: {len(downloaded_files)}/{total_months} file")
    print(f"   - Tổng dung lượng: {total_size:.1f} MB ({total_size/1024:.2f} GB)")
    print(f"   - Thư mục: {output_folder}")
    
    # Thống kê theo năm
    for year in range(2022, 2025):
        year_files = [f for f in downloaded_files if f"_{year}_" in f]
        year_size = sum(os.path.getsize(f) for f in year_files) / (1024*1024)
        print(f"   - Năm {year}: {len(year_files)} file ({year_size:.1f} MB)")

if failed_months:
    print(f"\n❌ Thất bại ({len(failed_months)} tháng):")
    for month in failed_months:
        print(f"   - {month}")
    print(f"\n💡 Có thể thử lại các tháng thất bại bằng cách chạy:")
    print(f"   download_monthly_chirps(year, month, {SCALE})")

print(f"\n📋 Danh sách file đã tải:")
for i, filepath in enumerate(downloaded_files, 1):
    filename = os.path.basename(filepath)
    size = os.path.getsize(filepath) / (1024*1024)
    print(f"   {i:2d}. {filename} ({size:.2f} MB)")

print(f"\n🔍 Sử dụng cell tiếp theo để kiểm tra chi tiết và phân tích dữ liệu!")
print(f"💡 Ứng dụng cho độ ẩm đất:")
print(f"   - CHIRPS cung cấp tổng lượng mưa hàng tháng từ dữ liệu daily")
print(f"   - Có thể tích hợp với Sentinel-1/2 để mô hình hóa độ ẩm đất")
print(f"   - Phù hợp cho nghiên cứu nông nghiệp và quản lý tài nguyên nước")
print(f"   - Dữ liệu được tính từ tổng lượng mưa daily trong tháng")

In [ ]:
# KIỂM TRA VÀ PHÂN TÍCH KẾT QUẢ CHIRPS
print("=== KIỂM TRA KẾT QUẢ TẢI CHIRPS ===")

def check_chirps_files():
    """Kiểm tra các file CHIRPS đã tải với thông tin chi tiết"""
    if not os.path.exists(output_folder):
        print("❌ Thư mục dữ liệu chưa tồn tại")
        return []
    
    files = [f for f in os.listdir(output_folder) if f.endswith('.tif')]
    
    if not files:
        print("📂 Thư mục trống - chưa có file nào")
        return []
    
    # Sắp xếp files theo tên
    files.sort()
    
    print(f"📊 Đã tải {len(files)} file CHIRPS:")
    print(f"📁 Thư mục: {output_folder}")
    
    total_size = 0
    monthly_stats = []
    
    for i, file in enumerate(files, 1):
        filepath = os.path.join(output_folder, file)
        size = os.path.getsize(filepath) / (1024*1024)  # MB
        total_size += size
        
        # Trích xuất thông tin từ tên file
        parts = file.split('_')
        if len(parts) >= 4:
            year = int(parts[2])
            month = int(parts[3])
            monthly_stats.append((year, month, size))
        
        print(f"   {i:2d}. {file} ({size:.2f} MB)")
    
    print(f"\n📈 Tổng quan:")
    print(f"   - Tổng số file: {len(files)}")
    print(f"   - Tổng dung lượng: {total_size:.1f} MB ({total_size/1024:.2f} GB)")
    print(f"   - Kích thước trung bình: {total_size/len(files):.2f} MB/file")
    
    # Thống kê theo năm
    for year in range(2022, 2025):
        year_files = [f for f in files if f"_{year}_" in f]
        year_size = sum(os.path.getsize(os.path.join(output_folder, f)) for f in year_files) / (1024*1024)
        print(f"   - Năm {year}: {len(year_files)} file ({year_size:.1f} MB)")
    
    return files

def analyze_chirps_file(filename):
    """Phân tích chi tiết file CHIRPS"""
    filepath = os.path.join(output_folder, filename)
    
    if not os.path.exists(filepath):
        print(f"❌ File không tồn tại: {filename}")
        return
    
    try:
        import rasterio
        import numpy as np
        
        with rasterio.open(filepath) as src:
            print(f"📄 Phân tích file: {filename}")
            print(f"   - Kích thước: {src.width} x {src.height} pixels")
            print(f"   - Số bands: {src.count}")
            print(f"   - CRS: {src.crs}")
            print(f"   - Bounds: {src.bounds}")
            print(f"   - Data type: {src.dtypes[0]}")
            
            # Đọc dữ liệu precipitation
            precipitation = src.read(1)
            
            # Loại bỏ nodata values
            valid_data = precipitation[precipitation != src.nodata]
            
            if len(valid_data) > 0:
                print(f"   - Lượng mưa (mm):")
                print(f"     • Trung bình: {np.mean(valid_data):.1f} mm")
                print(f"     • Min: {np.min(valid_data):.1f} mm")
                print(f"     • Max: {np.max(valid_data):.1f} mm")
                print(f"     • Std: {np.std(valid_data):.1f} mm")
                print(f"     • Median: {np.median(valid_data):.1f} mm")
                
                # Phân loại mức độ mưa
                dry_pixels = np.sum(valid_data < 50)
                normal_pixels = np.sum((valid_data >= 50) & (valid_data < 150))
                wet_pixels = np.sum(valid_data >= 150)
                
                total_pixels = len(valid_data)
                print(f"   - Phân loại mưa:")
                print(f"     • Khô hạn (<50mm): {dry_pixels/total_pixels*100:.1f}%")
                print(f"     • Bình thường (50-150mm): {normal_pixels/total_pixels*100:.1f}%")
                print(f"     • Ẩm ướt (>150mm): {wet_pixels/total_pixels*100:.1f}%")
            else:
                print(f"   - Không có dữ liệu hợp lệ")
            
    except ImportError:
        print(f"💡 Cài đặt rasterio và numpy để phân tích chi tiết:")
        print(f"   pip install rasterio numpy")
        file_size = os.path.getsize(filepath) / (1024*1024)
        print(f"📄 {filename}: {file_size:.2f} MB")
    except Exception as e:
        print(f"❌ Lỗi đọc file: {str(e)}")

def generate_chirps_summary():
    """Tạo báo cáo tóm tắt dữ liệu CHIRPS"""
    files = check_chirps_files()
    
    if not files:
        return
    
    print(f"\n📊 BÁO CÁO TÓM TẮT DỮ LIỆU CHIRPS")
    print(f"{'='*50}")
    
    try:
        import rasterio
        import numpy as np
        
        yearly_stats = {}
        
        for file in files:
            filepath = os.path.join(output_folder, file)
            parts = file.split('_')
            
            if len(parts) >= 4:
                year = int(parts[2])
                month = int(parts[3])
                
                if year not in yearly_stats:
                    yearly_stats[year] = {'months': [], 'total_precip': []}
                
                with rasterio.open(filepath) as src:
                    precipitation = src.read(1)
                    valid_data = precipitation[precipitation != src.nodata]
                    
                    if len(valid_data) > 0:
                        avg_precip = np.mean(valid_data)
                        yearly_stats[year]['months'].append(month)
                        yearly_stats[year]['total_precip'].append(avg_precip)
        
        # Hiển thị thống kê theo năm
        for year in sorted(yearly_stats.keys()):
            stats = yearly_stats[year]
            if stats['total_precip']:
                avg_annual = np.mean(stats['total_precip'])
                total_annual = np.sum(stats['total_precip'])
                months_count = len(stats['months'])
                
                print(f"\n🗓️  Năm {year} ({months_count} tháng):")
                print(f"   - Lượng mưa trung bình: {avg_annual:.1f} mm/tháng")
                print(f"   - Tổng lượng mưa: {total_annual:.1f} mm/năm")
                print(f"   - Tháng khô nhất: {min(stats['total_precip']):.1f} mm")
                print(f"   - Tháng ẩm nhất: {max(stats['total_precip']):.1f} mm")
        
        print(f"\n💡 Ứng dụng cho nghiên cứu độ ẩm đất:")
        print(f"   - Dữ liệu CHIRPS có thể được sử dụng làm input cho mô hình độ ẩm đất")
        print(f"   - Kết hợp với dữ liệu Sentinel-1/2 để cải thiện độ chính xác")
        print(f"   - Phân tích mối quan hệ giữa lượng mưa và độ ẩm đất theo thời gian")
        
    except ImportError:
        print("💡 Cài đặt rasterio và numpy để tạo báo cáo chi tiết")

def clean_chirps_folder():
    """Xóa tất cả file CHIRPS trong thư mục output"""
    if not os.path.exists(output_folder):
        print("❌ Thư mục không tồn tại")
        return
    
    files = [f for f in os.listdir(output_folder) if f.endswith('.tif')]
    
    if not files:
        print("📂 Thư mục đã trống")
        return
    
    confirm = input(f"⚠️  Xóa {len(files)} file CHIRPS? (y/N): ")
    if confirm.lower() == 'y':
        for file in files:
            os.remove(os.path.join(output_folder, file))
        print(f"🗑️  Đã xóa {len(files)} file")
    else:
        print("❌ Hủy bỏ")

# Kiểm tra kết quả hiện tại
files = check_chirps_files()

if files:
    print(f"\n💡 Sử dụng:")
    print(f"   - analyze_chirps_file('filename.tif') để phân tích chi tiết file")
    print(f"   - generate_chirps_summary() để tạo báo cáo tóm tắt")
    print(f"   - clean_chirps_folder() để xóa tất cả file")
    
    # Phân tích file đầu tiên
    if len(files) > 0:
        print(f"\n🔍 Phân tích file đầu tiên:")
        analyze_chirps_file(files[0])
        
    # Tạo báo cáo tóm tắt
    print(f"\n📋 Tạo báo cáo tóm tắt:")
    generate_chirps_summary()
else:
    print(f"\n⚠️  Chưa có file nào. Hãy chạy cell trước để tải dữ liệu.")